# Limpieza y transformación del Dataset

In [35]:
import pandas as pd
# Cargamos el dataset
df = pd.read_csv('../../data/dataset_original.csv', encoding='ISO-8859-1')
# Mostramos las primeras filas
df.head(4)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


- Filtramos líneas con precios que anulan las ventas


In [36]:
# Las facturas que tienen UnitPrice a 0, no aportan nada. FUERA!
df = df[df['UnitPrice'] != 0]
df[df['UnitPrice']==0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


- Filtramos las líneas con precios unitarios negativos

In [37]:
mask = df[['UnitPrice']] < 0
df = df[~mask.any(axis=1)]

print(f"Numero de 'UnitPrice' negativos antes: {mask.sum()['UnitPrice']}")
print(f"Numero de 'UnitPrice' negativos ahora: {(df[['UnitPrice']] < 0).sum()['UnitPrice']}")

Numero de 'UnitPrice' negativos antes: 2
Numero de 'UnitPrice' negativos ahora: 0


- Eliminamos las filas duplicadas

In [38]:
print(f"Numero de duplicados antes: {df.duplicated().sum()}")

df = df.drop_duplicates()

print(f"Numero de duplicados después: {df.duplicated().sum()}")

Numero de duplicados antes: 5263
Numero de duplicados después: 0


- Rellenamos los valores nulos de CustomerID

In [39]:
print(f"Número de 'CustomerID' nulos antes: {df['CustomerID'].isnull().sum()}")

# agrupamos por InvoiceNo , y usamos ffill y bfill para rellenar, los valores nulls que no se rellenen les da un valor nuevo unico
df['CustomerID'] = df.groupby(['InvoiceNo'])['CustomerID'].ffill().bfill()

print(f"Número de 'CustomerID' nulos ahora: {df['CustomerID'].isnull().sum()}")

Número de 'CustomerID' nulos antes: 132565
Número de 'CustomerID' nulos ahora: 0


- Rellenamos los valores nulos de Description

In [40]:
# tratamiento Description null
print(f"Número de 'Description' nulos antes: {df['Description'].isnull().sum()}")

# agrupamos por stockcode, y usamos ffill para rellenar con el anterior valor dentro del grupo, para los primer valor en null tambien uso bfill
df['Description'] = df.groupby(['StockCode'])['Description'].transform(lambda group: group.ffill().bfill())
df['Description'] = df['Description'].fillna('No description')

print(f"Número de 'Description' nulos ahora: {df['Description'].isnull().sum()}")

Número de 'Description' nulos antes: 0
Número de 'Description' nulos ahora: 0


- Detectamos y eliminamos los "outliers"

In [41]:
# Para los outliers, usamos el rango intercuartil (IQR)
total_sales = df['Quantity'] * df['UnitPrice']

# Para los outliers, usamos el rango intercuartil (IQR)
q1 = total_sales.quantile(0.25)
q3 = total_sales.quantile(0.75)
iqr = q3 - q1

# Ajuste de rango para 90%//95%
range=2.5
lower_limit = q1 - (range * iqr)
higher_limit = q3 + (range * iqr)

outliers_condition = (total_sales >= lower_limit) & (total_sales <= higher_limit)
filtered_df = df[outliers_condition]

# porcentaje filtarado
print(f"Porcentaje de datos retenidos: {len(filtered_df) / len(df) * 100:.2f}%")

df = filtered_df

Porcentaje de datos retenidos: 94.29%


# Transformación

- Calculamos las ventas totales para cada conjunto

In [42]:
df['TotalSales'] = df['Quantity'] * df['UnitPrice']

- Descomponemos las fechas en varias columnas para enriquecer los datasets

In [ ]:
def separate_date(df: pd.DataFrame):
  df['Year'] = df['InvoiceDate'].dt.year
  df['Month'] = df['InvoiceDate'].dt.month
  df['Day'] = df['InvoiceDate'].dt.day
  df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek
  df['Quarter'] = df['Month'].apply(
    lambda month: 
      1 if month in [1, 2, 3] else
      2 if month in [4, 5, 6] else
      3 if month in [7, 8, 9] else 
      4 # if no one chosen
  )
  df['Season'] = df['Month'].apply(
    lambda x: 
      'Winter' if x in [12, 1, 2] else
      'Spring' if x in [3, 4, 5] else
      'Summer' if x in [6, 7, 8] else 
      'Fall' # if no one chosen
  )
  return df


df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df = separate_date(df)

- Dividimos los datos en dos conjuntos (Entrenamiento y Prueba)

In [44]:
limit_date = pd.to_datetime("11/08/2011 00:00", format="%m/%d/%Y %H:%M")

df_prediction = df[df['InvoiceDate'] > limit_date]
df = df[df['InvoiceDate'] <= limit_date]

df_prediction.head(4)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSales,Year,Month,Day,DayOfWeek,Quarter,Season
447057,574943,17012D,ORIGAMI ROSE INCENSE/CANDLE SET,24,2011-11-08 07:52:00,0.85,13026.0,United Kingdom,20.4,2011,11,8,1,4,Fall
447058,574943,17012F,ORIGAMI SANDLEWOOD INCENSE/CAND SET,24,2011-11-08 07:52:00,0.85,13026.0,United Kingdom,20.4,2011,11,8,1,4,Fall
447059,574943,21636,MADRAS NOTEBOOK MEDIUM,12,2011-11-08 07:52:00,0.75,13026.0,United Kingdom,9.0,2011,11,8,1,4,Fall
447060,574943,21991,BOHEMIAN COLLAGE STATIONERY SET,12,2011-11-08 07:52:00,1.25,13026.0,United Kingdom,15.0,2011,11,8,1,4,Fall


- Guardamos el dataset de entrenamiento

In [45]:
df.to_csv('../../data/dataset_training.csv')

- Guardamos el dataset de prueba

In [46]:
df_prediction.to_csv('../../data/dataset_prediction.csv')